# General Notes:

This solution involves preinitializing the states and assigning them a value based on whether they represent a win, lose or other state.  There are 19683 possible board configurations (including invalid ones), which is very manageable.  Assigning the win condition of each state is important because the win condition check is fairly laborious in this implementation and would slow down training if you had to perform it every time you made a move.  

Another way to do this is to initialize new states upon encountering them for the first time (ie. if the current state is not found in the state dictionary). 

Exploration was removed during assessment

Performance:
- To keep things manageable, training was performed for only 1e5 epochs.
- Training with both performing random moves for 1e5 games produces a win rate of 97.0% once player 1 switches to greedy, or  84.4% if player 2 switches to greedy instead
- When both players are greedy, the outcome is 'Tie' most of the time, as expected
- Player 1 has the advantage when moving randomly


It is also important to note that player 1 and 2 value their states differently (ex. a 0 for player 1 is a 1 for player 2).
For two players, you can solve this problem by subtracting from 1.  For multiple players, a more sophisticated solution is required (ex. each player has their own state_values, you invert all the symbols so you act as though you're player one, etc.)

In [706]:
import numpy as np
import random

In [707]:
class Board():
    class _State():
        __slots__ = 'win_state', 'state_value', 'board'
            
        def __init__(self, win, value, board):
            self.win_state = win
            self.state_value = value
            self.board = board
            
            
        def update_value(self, value):
            if 0<=value<=1: 
                self.state_value = value
                return True
            
            else:
                return False
            
        def get_state_value(self):
            return self.state_value
            
        def win_check(self):
            return self.win_state
        
        def __repr__(self):
            return f'Win: {self.win_state}, Value: {self.state_value}'
    
    NUM_ROWS = 3
    NUM_COLS = 3
    BOARD_SIZE = NUM_ROWS * NUM_COLS
    PLAYERS = {'X', 'O'}
    BLANK_SPACE = '-'
    
    def __init__(self):
        self.board = ['-'] * (self.NUM_ROWS * self.NUM_COLS)
        self.rows = self.NUM_ROWS
        self.cols = self.NUM_COLS
        self.possible_values = self.PLAYERS | {self.BLANK_SPACE}
        self.player1 = 'X'
        self.player2 = 'O'
        self.state_values = {}
        
    def __repr__(self):
        self.print_board(self.board)
        return ""
        
    def clear_board(self):
        self.board = ['-'] * len(self.board)
        
    def print_board(self, board = None):
        if board is None: board = self.board
        for row in range(self.rows):
            output_row = board[self.cols*row: self.cols*(row+1)]
            print (output_row)
            
    def check_win(self, char, board = None):
        if board is None: board = self.board
        #Check Rows
        for row in range(self.rows):
            row_check = board[self.cols*row: self.cols*(row+1)]
            if row_check[0] == char and row_check.count(row_check[0])==self.rows: return True
        
        #Check Columns
        for column in range(self.cols):
            col_check = [board[self.cols*x+column] for x in range(self.rows)]
            if col_check[0] == char and col_check.count(col_check[0])==self.cols: return True
            
        #Check Diagonals
        #for dir in [1, -1]:
        diag_check = [board[self.cols*x+x] for x in range(self.rows)]
        if diag_check[0] == char and diag_check.count(diag_check[0])==self.rows: return True
        
        diag_check = [board[self.cols*(x+1)-x-1] for x in range(self.rows)]
        if diag_check[0] == char and diag_check.count(diag_check[0])==self.rows: return True
        
        return False
    
    
    def create_state(self, win, value, board):
        return self._State(win, value, board)
    
    def _rec_state_values(self, index, state, states):
        #if you have reached the 
        if index == self.rows*self.cols:
            win = self.check_win(self.player1, state)
            lose = self.check_win(self.player2, state)
            key = ''.join(state)
            
                       
            if not (win or lose):  
                states[key] = self.create_state(False, 0.5, state)
            else: states[key] = self.create_state(True, win*1, state)
            
        else:
            for char in self.possible_values:
                state[index] = char
                self._rec_state_values(index + 1, state, states)
                state[index] = self.BLANK_SPACE
                
    
    
    def initialize_state_values(self):
        states = {}
        state = [self.BLANK_SPACE]*(self.rows*self.cols)
        index = 0
        self._rec_state_values(index, state, states)
        self.state_values = states
        
    
    
    def move(self, player, position):
        if player not in self.PLAYERS: return False
        if not self.board[position] == self.BLANK_SPACE: return False
        
        self.board[position] = player
        return True
    
    def get_choices(self):
        return [x for x in range(self.cols*self.rows) if self.board[x] == self.BLANK_SPACE]
    
    def get_state(self, state_key = None):
        if state_key is None: state_key = self.board
        state_key = ''.join(state_key)
        if state_key in self.state_values: return self.state_values[state_key]
        else: return False
        
    
    
    def random_move(self, player):
        choices = self.get_choices()
        #print ('Choices are', choices)
        if len(choices) == 0: return False
        else: return self.move(player, random.choice(choices))
        
        
        
    def greedy_move(self, player, invert = False):
        choices = self.get_choices()
        board_copy = self.board.copy()
        filtered_choices = []
        for position in choices:
            board_copy[position] = player #set the new state
            state_value = self.get_state(board_copy).get_state_value()
            if invert: state_value = 1-state_value  #Do this if you are player 2
            
            #Mave a list of the positions with the highest state values
            if len(filtered_choices) == 0: filtered_choices = [(position, state_value)]
            elif state_value > filtered_choices[0][1]: filtered_choices = [(position, state_value)]
            elif state_value == filtered_choices[0][1]: filtered_choices.append((position, state_value))
            board_copy[position] = '-'  #return to the original state
            
        #Choose randomly among the equivalent best options
        return self.move(player, random.choice(filtered_choices)[0])
        
        

        
        
    
    #for testing only
    def input_board(self, array):
        self.board = array

In [708]:
class Game():  
    class Player():
        def __init__(self, token, strategy):
            self.token = token
            self.strategy = strategy

        def get_token(self):
            return self.token
        
        def get_strategy(self):
            return self.strategy
    
        def change_strategy(self, strategy):
            self.strategy = strategy
    
    def __init__(self, board, player1_strat = None, player2_strat = None):
        self.board = board
        self.num_players = 2
        self.current_board_state = board.get_state(self.board.board)
        self.players = [self.Player('X', player1_strat), self.Player('O', player2_strat)]
        self.train_mode = False
        self._moves = 0     #Failsafe to avoid infinite loops
        self._max_moves = self.board.rows * self.board.cols
        self.output_moves = True
        
        self.scores = [0]*(self.num_players + 1)
        

    def make_move(self, player, strategy):
        if strategy == None or strategy == 'Random':
            #print('Doing a random move')
            #return partial(random_move, player = player)
            return self.random_move(player)
        
        elif strategy == 'Greedy':
            invert = player == self.players[1].get_token() #Are you player 2??
            if self.train_mode: return self.greedy_move(player, invert)
            else: return self.greedy_move(player, invert, exploration_chance = 0.0)
        
        else:
            print('Unknown strategy', strategy)
            return False
        
    def reset_scores(self):
        self.scores = [0]*len(self.scores)
    
    def random_move(self, player):
        self.board.random_move(player)
        return True
    
    def greedy_move(self, player, invert = False, exploration_chance = 0.1):
        if random.random() > exploration_chance:
            self.train_mode = True
            self.board.greedy_move(player, invert)
        else:
            self.train_mode = False #Don't train if you make an exploratory move
            self.board.random_move(player)
        return True
    
    
    def train(self, num_games, train_rate = 0.05):
        self.train_mode = True
        self.output_moves = False
        self.train_rate = train_rate
        for x in range(num_games):
            self.play()     
        self.train_mode = False
        
        
    def assess(self, num_games):
        self.reset_scores()
        self.train_mode, self.output_moves = False, False
        for x in range(num_games):
            self.play()
        print('Final Scores are:', self.scores)
            
    def play_single(self):
        self.train_mode = False
        self.output_moves = True
        self.play()
    
    def change_strategy(self, player_number, strategy):
        player = self.players[player_number]
        player.change_strategy(strategy)
    
    def play(self):
        current_state = self.current_board_state
        self._moves = 0
        self.board.clear_board()       
        
        while not current_state.win_check() and self._moves < self._max_moves:
            #keep Note of the current state for training)
            previous_state = current_state
            
            
            
            current_player = self.players[(self._moves)%self.num_players]

            token, strategy = current_player.get_token(), current_player.get_strategy()
            if self.make_move(token, strategy):pass
            else: print('Could not make a move')
                
            current_state = self.board.get_state(self.board.board)
            self._moves += 1
            
            
            if self.train_mode:
                previous_value = previous_state.get_state_value()
                current_value = current_state.get_state_value()
                previous_state.update_value(previous_value + self.train_rate*(current_value - previous_value))
            
            if self.output_moves:
                print('Current Player is...', current_player.token, 'with the strategy', current_player.get_strategy())
                print(self.board)
            
        if self.output_moves:    
            if current_state.win_check(): print (f'Player {current_player.token} wins!!!')
            else: print('Tie!!')
                
        if current_state.win_check(): self.scores[(self._moves - 1)%self.num_players] += 1
        else: self.scores[-1] += 1
        

## Training and Evalutation

In [709]:
#Train using random moves

bb = Board()
bb.initialize_state_values()
game1 = Game(bb, 'Random', 'Random')
game1.train(int(1e5), 0.5)

print ("When training using only random moves, the results are...")
game1.assess(1000)



When training using only random moves, the results are...
Final Scores are: [564, 300, 136]


In [710]:
#Train using random moves, play as greedy (player 1)

bb = Board()
bb.initialize_state_values()
game2 = Game(bb, 'Random', 'Random')
game2.train(int(1e5), 0.5)

game2.change_strategy(0, 'Greedy')

print ("When training using only random moves but playing as greedy, the results are...")
game2.assess(1000)

When training using only random moves but playing as greedy, the results are...
Final Scores are: [973, 11, 16]


In [711]:
#Train using random moves, play as greedy (player 2)

bb = Board()
bb.initialize_state_values()
game3 = Game(bb, 'Random', 'Random')
game3.train(int(1e5), 0.5)

game3.change_strategy(1, 'Greedy')

print ("When training using only random moves but playing as greedy, the results are...")
game3.assess(1000)

When training using only random moves but playing as greedy, the results are...
Final Scores are: [101, 831, 68]


In [712]:
#Train using random moves, both play as greedy

bb = Board()
bb.initialize_state_values()
game4 = Game(bb, 'Random', 'Random')
game4.train(int(1e5), 0.5)

game4.change_strategy(0, 'Greedy')
game4.change_strategy(1, 'Greedy')




print ("When training using only random moves but playing as greedy, the results are...")
game4.assess(1000)

When training using only random moves but playing as greedy, the results are...
Final Scores are: [291, 132, 577]


In [713]:
#Train using random moves, both play as greedy

bb = Board()
bb.initialize_state_values()
game5 = Game(bb, 'Random', 'Random')
game5.train(int(1e5), 0.5)

game5.change_strategy(0, 'Greedy')
game5.change_strategy(1, 'Greedy')

game5.train(int(1e6), 0.05)


print ("When training using only random moves but playing as greedy, the results are...")
game5.assess(1000)

When training using only random moves but playing as greedy, the results are...
Final Scores are: [152, 96, 752]
